In [ ]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.primitives import Estimator, Sampler
from qiskit.quantum_info import SparsePauliOp
import matplotlib.pyplot as plt
from qiskit.circuit.library import EfficientSU2,RealAmplitudes

In [ ]:
n_qubits=5

L=2

In [ ]:
sampler = Sampler()
estimator = Estimator()
ob = SparsePauliOp.from_list([("Z"+"I"*(n_qubits-1), 1)])
print(ob)

In [ ]:
ob_list1 = []
for a in ['X','Y','Z','I']:
    op = SparsePauliOp.from_list([(a+"I"*(n_qubits-1), 1)])
    ob_list1.append(op)
            
ob_list2 = []
for a in ['X','Y','Z','I']:
    for b in ['X','Y','Z','I']:
        pauli = a+b
        if pauli != 'II':
            op = SparsePauliOp.from_list([(pauli+"I"*(n_qubits-2), 1)])
            ob_list2.append(op)
                
ob_list3 = []
for a in ['X','Y','Z','I']:
    for b in ['X','Y','Z','I']:
        for c in ['X','Y','Z','I']:
            pauli = a+b+c
            if pauli != 'III':
                op = SparsePauliOp.from_list([(pauli+"I"*(n_qubits-3), 1)])
                ob_list3.append(op)

                    
ob_list4 = []
for a in ['X','Y','Z','I']:
    for b in ['X','Y','Z','I']:
        for c in ['X','Y','Z','I']:
            for d in ['X','Y','Z','I']:
                pauli = a+b+c+d
                if pauli != 'IIII':
                    op = SparsePauliOp.from_list([(pauli+"I"*(n_qubits-4), 1)])
                    ob_list4.append(op)

ob_list5 = []
for a in ['X','Y','Z','I']:
    for b in ['X','Y','Z','I']:
        for c in ['X','Y','Z','I']:
            for d in['X','Y','Z','I']:
                for e in ['X','Y','Z','I']:
                    pauli = a+b+c+d+e
                    if pauli != 'IIII':
                        op = SparsePauliOp.from_list([(pauli+"I"*(n_qubits-5), 1)])
                        ob_list5.append(op)

In [ ]:
deck = list(range(0, len(ob_list5)))
np.random.shuffle(deck)
index=deck[0:len(ob_list3)]

ob_list3_random = []
for i in range(len(ob_list5)):
    
    if i in index:
        ob_list3_random.append(ob_list5[i])

In [ ]:
#Create datasets:

n_reps_init=5

# qc_init = QuantumCircuit(n_qubits,len(measured_bits))
# ansatz_init =  RealAmplitudes(num_qubits=n_qubits,reps=n_reps_init,parameter_prefix='w')
# qc_init.compose(ansatz_init, inplace=True)  
# qc_init.measure(measured_bits,list(range(len(measured_bits))))

qc_init = QuantumCircuit(n_qubits)
ansatz_init =  RealAmplitudes(num_qubits=n_qubits,reps=n_reps_init,parameter_prefix='w')
qc_init.compose(ansatz_init, inplace=True)  

qc_all = QuantumCircuit(n_qubits)
ansatz_init2 =  RealAmplitudes(num_qubits=n_qubits,reps=n_reps_init,parameter_prefix='w2')
qc_all.compose(ansatz_init2, inplace=True)  
qc_all.barrier()
ansatz =  RealAmplitudes(num_qubits=n_qubits,reps=L,parameter_prefix='a')
qc_all.compose(ansatz,inplace=True)
VQA_params = 2*np.pi*np.random.rand(qc_all.num_parameters-qc_init.num_parameters)

param_init_list = []
info1_list = []
info2_list = []
info3_list = []
info3_list_random = []
info4_list = []
output_list = []

for i in range(3000):
    if i%50==0:
        print(i," samples prepared")
    param_init=2*np.pi*np.random.rand(qc_init.num_parameters)
    params_all = np.concatenate([VQA_params,param_init])
    
    info1 = estimator.run([qc_init]*len(ob_list1),observables=ob_list1,parameter_values=[param_init]*len(ob_list1)).result().values
    info2 = estimator.run([qc_init]*len(ob_list2),observables=ob_list2,parameter_values=[param_init]*len(ob_list2)).result().values
    info3 = estimator.run([qc_init]*len(ob_list3),observables=ob_list3,parameter_values=[param_init]*len(ob_list3)).result().values
    info3_random = estimator.run([qc_init]*len(ob_list3_random),observables=ob_list3_random,parameter_values=[param_init]*len(ob_list3_random)).result().values
    info4 = estimator.run([qc_init]*len(ob_list4),observables=ob_list4,parameter_values=[param_init]*len(ob_list4)).result().values
   
    output = estimator.run(qc_all,observables=ob,parameter_values=params_all).result().values[0]

    param_init_list.append(param_init)
    info1_list.append(info1)
    info2_list.append(info2)
    info3_list.append(info3)
    info3_list_random.append(info3_random)
    info4_list.append(info4)
    output_list.append(output)
    
param_init_list = np.array(param_init_list)
info1_list = np.array(info1_list)
info2_list = np.array(info2_list)
info3_list = np.array(info3_list)
info3_list_random = np.array(info3_list_random)
info4_list = np.array(info4_list)
output_list = np.array(output_list)
    

In [ ]:
qc_all.decompose().draw('mpl')

In [ ]:
#turning into classification problem:
output_list=np.floor(output_list)+1

In [ ]:
np.save("dataset/info1_list_L{}".format(L), info1_list)
np.save("dataset/info2_list_L{}".format(L), info2_list)
np.save("dataset/info3_list_L{}".format(L), info3_list)
np.save("dataset/info3_list_L{}_random".format(L), info3_list_random)
np.save("dataset/info4_list_L{}".format(L), info4_list)
np.save("dataset/output_list_L{}".format(L), output_list)

In [ ]:
#classical NN:
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(
       info1_list, output_list, test_size=0.1, random_state=42)
X_train2, X_test2, y_train2, y_test2 = train_test_split(
       info2_list, output_list, test_size=0.1, random_state=42)
X_train3, X_test3, y_train3, y_test3 = train_test_split(
       info3_list, output_list, test_size=0.1, random_state=42)

X_train3_random, X_test3_random, y_train3_random, y_test3_random = train_test_split(
       info3_list_random, output_list, test_size=0.1, random_state=42)

X_train4, X_test4, y_train4, y_test4 = train_test_split(
       info4_list, output_list, test_size=0.1, random_state=42)

In [ ]:
def train(X_train,y_train,regularization = 0.01,loss_fn = tf.keras.losses.BinaryCrossentropy()):
    n_neurons=100
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(n_neurons, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(regularization )),
        tf.keras.layers.Dense(n_neurons, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(regularization )),
        tf.keras.layers.Dense(n_neurons, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(regularization )),
        tf.keras.layers.Dense(n_neurons, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(regularization )),

        tf.keras.layers.Dense(1, activation='sigmoid'),  
    ])

    model.compile(optimizer="adam",
                  loss=loss_fn,
                 metrics=['BinaryCrossentropy','accuracy'])

    best_model_path="bestmodel"
    save_best_model = tf.keras.callbacks.ModelCheckpoint(best_model_path, monitor='val_loss', 
                                      save_best_only=True, save_weights_only=True)

    history = model.fit(X_train, y_train, validation_split = 0.2 ,epochs=100,callbacks=[save_best_model])
    model.load_weights(best_model_path)
    
    return model,history

In [ ]:
model1,history1=train(X_train1,y_train1)

In [ ]:
model2,history2=train(X_train2,y_train2)

In [ ]:
model3,history3=train(X_train3,y_train3)

In [ ]:
model3_random,history3_random=train(X_train3_random,y_train3_random)

In [ ]:
model4,history4=train(X_train4,y_train4)

In [ ]:

LabelSize=30
plt.figure(1)
plt.figure(figsize=(16,9))
plt.plot(history1.history['loss'],label = "1qubit info (train)")
plt.plot(history2.history['loss'],label = "2qubit info (train)")
plt.plot(history3.history['loss'],label = "3qubit info (train)")
plt.plot(history3_random.history['loss'],label = "3qubit info random (train)")
plt.plot(history4.history['loss'],label = "4qubit info (train)")

plt.yscale('log')
plt.title('Training Loss (L={})'.format(L),fontsize=LabelSize)
plt.ylabel('Loss',fontsize=LabelSize)
plt.xlabel('Epoch',fontsize=LabelSize)
plt.tick_params(axis='both', which='major', labelsize=LabelSize)
plt.legend(fontsize=0.5*LabelSize)
plt.savefig("plots/Training_loss_L{}".format(L))
plt.show()


In [ ]:
plt.figure(1)
plt.figure(figsize=(16,9))
plt.plot(history1.history['val_loss'],label = "1qubit info (val)")
plt.plot(history2.history['val_loss'],label = "2qubit info (val)")
plt.plot(history3.history['val_loss'],label = "3qubit info (val)")
plt.plot(history3_random.history['val_loss'],label = "3qubit info random (val)")
plt.plot(history4.history['val_loss'],label = "4qubit info (val)")
plt.yscale('log')

plt.title('Validation Loss (L={})'.format(L),fontsize=LabelSize)
plt.ylabel('Loss',fontsize=LabelSize)
plt.xlabel('Epoch',fontsize=LabelSize)
plt.tick_params(axis='both', which='major', labelsize=LabelSize)
plt.legend(fontsize=0.5*LabelSize)
plt.savefig("plots/Validation_loss_L{}".format(L))
plt.show()

In [ ]:
plt.figure(1)
plt.figure(figsize=(16,9))
plt.plot(history1.history['accuracy'],label = "1qubit info (train)")
plt.plot(history2.history['accuracy'],label = "2qubit info (train)")
plt.plot(history3.history['accuracy'],label = "3qubit info (train)")
plt.plot(history3_random.history['accuracy'],label = "3qubit info random (train)")
plt.plot(history4.history['accuracy'],label = "4qubit info (train)")


#plt.yscale('log')
plt.title('Training Accuracy (L={})'.format(L),fontsize=LabelSize)
plt.ylabel('accuracy',fontsize=LabelSize)
plt.xlabel('Epoch',fontsize=LabelSize)
plt.tick_params(axis='both', which='major', labelsize=LabelSize)
plt.legend(fontsize=LabelSize)
plt.savefig("Training_acc_L{}".format(L))
plt.show()

In [ ]:
plt.figure(1)
plt.figure(figsize=(16,9))
plt.plot(history1.history['val_accuracy'],label = "1qubit info (val)")
plt.plot(history2.history['val_accuracy'],label = "2qubit info (val)")
plt.plot(history3.history['val_accuracy'],label = "3qubit info (val)")
plt.plot(history3_random.history['val_accuracy'],label = "3qubit info random (val)")
plt.plot(history4.history['val_accuracy'],label = "4qubit info (val)")
#plt.yscale('log')
plt.title('Validation Accuracy (L={})'.format(L),fontsize=LabelSize)
plt.ylabel('accuracy',fontsize=LabelSize)
plt.xlabel('Epoch',fontsize=LabelSize)
plt.tick_params(axis='both', which='major', labelsize=LabelSize)
plt.legend(fontsize=LabelSize)
plt.savefig("validation_acc_L{}".format(L))
plt.show()